In [1]:
import ncem
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import anndata as ad
sc.settings.set_figure_params(dpi=80)

2024-06-11 17:10:23.195433: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-11 17:10:26.894930: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-11 17:10:33.236204: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 17:11:32.993043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# NCEM

NCEM github repo: https://ncem.readthedocs.io/en/latest/

Geome Linear NCEM: https://github.com/theislab/geome/blob/main/docs/notebooks/4_linear_NCEM_graph.ipynb

In [7]:
data_path_base = "/lustre/groups/ml01/datasets/2024_spatial_ccc_experiments_francesca.drummer/dataset.h5ad"

## Model training

https://github.com/theislab/ncem_tutorials/blob/main/tutorials/model_tutorial_interactions.ipynb

In [8]:
data_set = 'zhuang'
data_path = data_path_base + '/zhuang/'
log_transform = False
use_domain = True
scale_node_size=False
merge_node_types_predefined = True
covar_selection = []
output_layer='linear'

In [9]:
# define parameters
model_class = 'interactions'
optimizer = 'adam'
domain_type = 'patient'

learning_rate = 0.05
l1 = 0.
l2 = 0.

batch_size = 64
radius = 100
n_eval_nodes = 10

gs_id = f"tutorial_{model_class}_{radius}_{data_set}_{domain_type}"

In [10]:
ncv = 3
epochs = 2000 if "tutorial" not in gs_id else 10 
epochs_warmup = 0
max_steps_per_epoch = 20
patience = 100
lr_schedule_min_lr = 1e-10
lr_schedule_factor = 0.5
lr_schedule_patience = 50
val_bs = 16
max_val_steps_per_epoch = 10
shuffle_buffer_size = None

feature_space_id = "standard"
cond_feature_space_id = "type"

use_covar_node_label = False
use_covar_node_position = False
use_covar_graph_covar = False

In [11]:
trainer = ncem.train.TrainModelInteractions()

In [12]:
trainer.init_estim(log_transform=log_transform)

In [13]:
trainer.estimator.get_data(
    data_origin=data_set,
    data_path=data_path,
    radius=radius,
    graph_covar_selection=covar_selection,
    node_label_space_id=cond_feature_space_id,
    node_feature_space_id=feature_space_id,
    # feature_transformation=transformation_dict[transform_key],
    use_covar_node_position=use_covar_node_position,
    use_covar_node_label=use_covar_node_label,
    use_covar_graph_covar=use_covar_graph_covar,
    # hold_out_covariate=hold_out_covariate,
    domain_type=domain_type,
    # merge_node_types_predefined=merge_node_types_predefined,
)

UnboundLocalError: local variable 'DataLoader' referenced before assignment

## Analysis

In [2]:
input_path = "/lustre/groups/ml01/datasets/2024_spatial_ccc_experiments_francesca.drummer/dataset.h5ad"
output_path = "/lustre/groups/ml01/datasets/2024_spatial_ccc_experiments_francesca.drummer/experiments"

In [3]:
adata = ad.read_h5ad(input_path)
adata

AnnData object with n_obs × n_vars = 37068 × 1122
    obs: 'brain_section_label', 'brain_section_label_right', 'feature_matrix_label', 'donor_label', 'donor_genotype', 'donor_sex', 'cluster_alias', 'x', 'y', 'z', 'subclass_confidence_score', 'cluster_confidence_score', 'high_quality_transfer', 'neurotransmitter', 'class', 'subclass', 'supertype', 'cluster', 'neurotransmitter_color', 'class_color', 'subclass_color', 'supertype_color', 'cluster_color', 'dataset_id', 'sample_id', 'donor_id', 'condition_id', 'tissue', 'organism', 'assay_ontology', 'assay', 'celltype', 'fov'
    var: 'gene_symbol', 'name', 'mapped_ncbi_identifier'
    uns: 'log1p'
    obsm: 'spatial'
    layers: 'counts', 'normalized'

In [4]:
interpreter = ncem.interpretation.interpreter.InterpreterInteraction()